In [13]:
import boto3
import pandas as pd
import numpy as np
import sys
from io import StringIO
import surprise
from surprise import accuracy
from sklearn.model_selection import train_test_split

In [2]:
sys.path.insert(1, '../src')

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%reload_ext autoreload

In [5]:
s3 = boto3.client('s3')

In [6]:
df_all_15 = pd.read_csv("s3://recommender-system-amazon/df_all_15_text.csv")

In [8]:
df_all_15 = df_all_15[~(df_all_15['title_nlp'].isna())]
df_all_15 = df_all_15[~(df_all_15['item_text'].isna())]
df_all_15 = df_all_15[~(df_all_15['reviewText'].isna())]

In [9]:
df_all_15.columns

Index(['Unnamed: 0', 'category', 'description', 'title', 'also_buy', 'image_x',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'price', 'asin',
       'overall', 'verified', 'reviewText', 'summary', 'unixReviewTime',
       'style', 'item_text', 'title_nlp'],
      dtype='object')

In [10]:
df_collab = df_all_15[["asin","overall"]].reset_index()

In [11]:
df_collab.head()

,index,asin,overall
0,0,0899332757,5.0
1,1,0899332757,4.0
2,2,0899332757,5.0
3,3,0899333257,5.0
4,4,0899333257,5.0


In [12]:
df_collab.shape

(369155, 3)

In [13]:
df_collab.describe()

,index,overall
count,369155.000000,369155.000000
mean,185148.438423,4.447806
std,106886.985863,0.965468
min,0.000000,1.000000
25%,92589.500000,4.000000
50%,185166.000000,5.000000
75%,277729.500000,5.000000
max,370251.000000,5.000000


In [15]:
# X_Train,X_holdout = train_test_split(df_collab, test_size=0.2)

In [16]:
reader = surprise.Reader(rating_scale=(1,5))

In [17]:
# data = surprise.Dataset.load_from_df(X_Train,reader)

data = surprise.Dataset.load_from_df(df_collab,reader)
# holdout = surprise.Dataset.load_from_df(X_holdout,reader)

In [16]:
kSplit = surprise.model_selection.split.KFold(n_splits=1, shuffle=True)

In [22]:
sim_options = sim_options = {'name': 'cosine','user_based': False}

collabKNN = surprise.KNNBasic(k=10,min_k=1,sim_options=sim_options,verbose=True) #try removing sim_options. 
# collabKNN = surprise.SVD()
trainset = data.build_full_trainset()
collabKNN.fit(trainset)
testset = trainset.build_testset()
predictions = collabKNN.test(testset)
accuracy.rmse(predictions, verbose=True)

# rmseKNN = []



# for trainset, testset in kSplit.split(data): #iterate through the folds.
#     collabKNN.fit(trainset)
#     predictionsKNN = collabKNN.test(testset)
#     rmseKNN.append(surprise.accuracy.rmse(predictionsKNN,verbose=True))#get root means squared 

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000


0.0

In [18]:
sm = collabKNN.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
sm.shape

(56351, 56351)